**NOTE** `phimats` environment should be used as kernel

In [33]:
import numpy as np
import h5py

from PreProcessing import PhysicsConfig, MeshConfig, PreProcessing as PP

from MeshManager import MeshManager

from BoundaryConditions import *

from PostProcessing import WriteXDMF

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Simulation data

In [34]:
# Simulation name
SimulName = "TDSHT990"
# Simulation type
SimulType = "2PhaseTrapping"
# Element sets
nElementSets = 1
# Number of steps
nSteps = 50

### Read mesh file 

In [35]:
# Element name
elementName = "triangle"  		# meshio compatible element name
mesh = MeshManager("../RVE_HT990.msh", elementName)
mesh.WriteMesh(SimulName)

Reading mesh: RVE_HT990.msh...

Mesh consistency verified: All domain elements are <triangle>.

PHIMATS MESH INITIATION
Detected 1 Material Sets. Follow this order in your Input Data:
  [Index 1] -> Physical Group: 'RVE'

------------------------------
MESH SUMMARY
------------------------------
File Name:      RVE_HT990.msh
Element Type:   triangle
Dimension:      2D
Node Order:     1
Total Nodes:    289263
Total Elements: 577013
Cell Blocks:
  - line: 1511 items
  - triangle: 577013 items
------------------------------



In [36]:
# Create the config object
meshConfig = MeshConfig(
    nTotNodes=mesh.get_nTotNodes(),
    nTotElements=mesh.get_nTotElements(),
    nDim=mesh.get_nDim(),
    materialNames=mesh.getMaterialNames(),
)

### Read phase-field data

In [37]:
# Path to the phase-field RVE data
fh5 = h5py.File("../HT990.rve.hdf5", "r")

gPhi_jj = fh5["gPhi_jj"][()]   
gPhi_ij = fh5["gPhi_ij"][()]   
gPhi_ii = fh5["gPhi_ii"][()]   
phi_j   = fh5["phi_j"][()]   

fh5.close()

### Diffusivity parameters

In [38]:
T = 281                    # Temperature [K] 8C

R = 8.31446261815324       # Universal gas constant [J/mol.K]

dt = 25                    # Time increment [s]

# Lattice diffusivity m²/s
D01 = 8.45e-6; DQ1 = 37700  
DL = D01*np.exp(-DQ1/(R*T)) 

# High solubility phase diffusivity
D02 = 5.76e-7; DQ2 = 49620 
DT = D02*np.exp(-DQ2/(R*T))

print("DL: ", DL)
print("DT: ", DT)
print("DL/DT: ", DL/DT)

DL:  8.298525128712058e-13
DT:  3.4420742950361795e-16
DL/DT:  2410.9081958746133


### Solubility parameters

In [39]:
Ejj       = 2000
Eij       = 5000
Eii       = 10000

Ej        = 12000

zeta_ii = 4*Eii
print("Zii: ", np.exp(zeta_ii/(4*R*T)))

zeta_ij = 4*Eij  # What actually happes is 4*(Eij+Ej) due to diffuse interface
print("Zij: ", np.exp(zeta_ij/(4*R*T)))

zeta_jj = 4*Ejj   # What actually happes is 4*(Ejj+Ej) due to diffuse interface
print("Zjj: ", np.exp(zeta_jj/(4*R*T)))

zeta_j = Ej
print("Zj: ", np.exp(zeta_j/(R*T)))

Zii:  72.25163696760185
Zij:  8.500096291666457
Zjj:  2.3537998900397787
Zj:  170.06589514953538


In [40]:
# Material `dict`
Materials = {
    "RVE" : {
		"D0x1" : D01,
		"D0y1" : D01,
		"DQx1" : DQ1,
		"DQy1" : DQ1,
		"D0x2" : D02,
		"D0y2" : D02,
		"DQx2" : DQ2,
		"DQy2" : DQ2,
		"zeta_j"   : zeta_j,
		"zeta_jj"  : zeta_jj,
		"zeta_ij"  : zeta_ij,
		"zeta_ii"  : zeta_ii,
    }
}

### Apply boundary conditions

In [41]:
exitNods = []
conBCs = []

# Entry con
conBCs.extend(AssignDirichletBC(mesh.getNodesByGroup("left"), dof=0, value=0))

# Exit con
conBCs.extend(AssignDirichletBC(mesh.getNodesByGroup("right"), dof=0, value=0))

In [42]:
conBCs = [[row[0], row[-1]] for row in conBCs]

### Initialize simulation object

In [43]:
# Create the config object
diffPhysConfig = PhysicsConfig(
    SimulName = SimulName,
    PhysicsType = "Transport",
    PhysicsCategory = SimulType,
    nSteps    = nSteps,
    presBCs=conBCs,
    exitNodes = exitNods,
    dt = dt,
    Temperature = T,
)

In [44]:
Duplex_TDS = PP(diffPhysConfig, meshConfig, Materials)


PHIMATS: TDSHT990 | Transport (2PhaseTrapping)
  Nodes/Elems : 289263 / 577013
  Total DOFs  : 289263
  BCs Count   : 460



In [45]:
Duplex_TDS.WriteInputFile()

  Input file initialized: TDSHT990.diff.in.hdf5


In [46]:
Duplex_TDS.WriteOutputFile(OVERWRITE=True, AVCON=True, TDS=True)

  Output file initialized: TDSHT990.diff.out.hdf5


### Generate xdmf visualization file

In [47]:
WriteXDMF(SimulName, # Base simulation name
          elementName, # Element name
          nSteps+1, # Number of steps 
          components=["diff"],
		  nDim=2,
    	  TDS=True
		  )